In [51]:
import os
import numpy as np
import pandas as pd

# Dataset Collection
Assignee: 
+ Wen Hao: Build scraper module to gather images of faces
+ Gin, Xiaoxin: Data cleaning. Remove images with inaccuracy label manually.

Description:

# Data Pre-processing
Assignee: Wen Hao

Description: Pre-process the images gathered. Standard image size, grayscale, etc.

# Data Exploration
Assignee: Gin, Xiaoxin

In [72]:
IMAGE_FOLDER = # replace it with your own path

totalImages = {}
for subfolder in os.listdir(IMAGE_FOLDER):
    dir = IMAGE_FOLDER + '/' + subfolder
    if os.path.isdir(dir):
        totalImages[subfolder] = len(os.listdir(dir))

pd.DataFrame(totalImages.items(), columns = ['category', '# of sample'])

,category,# of sample
0,neutral man face,241
1,sad woman face,848
2,angry woman face,753
3,disgusted woman face,746
4,happy man face,192
5,surprised woman face,851
6,sad man face,359
7,scared man face,394
8,angry man face,331
9,disgusted man face,514


In [73]:
print(sum(totalImages.values()), "files in total.")

7981 files in total.


# Model Development, Improvement & Evaluation